# In this notebook we will train the random forest model

In [ ]:
import numpy as np, pandas as pd, time, sys, warnings
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             make_scorer, accuracy_score, roc_auc_score)
warnings.filterwarnings("ignore")
np.random.seed(42)

# ──────────────────────────────────────────────────────────────
# CONFIG
# ──────────────────────────────────────────────────────────────
CSV_FILE = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
TIME_COLUMN, TARGET_COL = "timestamp", "target"
START_DATE, TEST_FRAC = "2018-01-01", 0.20 
DROP_COLS = [ 
    'open', 'high', 'low', 'high_low', 'high_close', 'low_close', 'typical_price',
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'rsi_oversold', 'rsi_overbought', 'stoch_overbought',
    'stoch_oversold', 'cci_overbought', 'cci_oversold', 'near_upper_band',
    'near_lower_band', 'overbought_reversal', 'oversold_reversal',
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'ema7_above_ema21', 'macd_rising',
    'bollinger_upper', 'bollinger_lower', 'bullish_scenario_1',
    'bullish_scenario_5', 'bearish_scenario_1'
]

# ──────────────────────────────────────────────────────────────
# LOAD DATA
# ──────────────────────────────────────────────────────────────
print("📊 Loading 4H Bitcoin data...")
if not CSV_FILE.exists():
    sys.exit(f"❌ File not found: {CSV_FILE}")

df = pd.read_csv(CSV_FILE, parse_dates=[TIME_COLUMN]).set_index(TIME_COLUMN).sort_index()
df = df.loc[START_DATE:].copy()

# Verify target column exists
if TARGET_COL not in df.columns:
    sys.exit(f"❌ Target column '{TARGET_COL}' not found!")

X = df.drop(columns=[col for col in DROP_COLS if col in df.columns] + [TARGET_COL], errors="ignore")
y = df[TARGET_COL]

# Chronological split (IMPORTANT: maintains time order)
split = int(len(df) * (1 - TEST_FRAC))
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

print(f"   📅 Date range: {df.index.min()} to {df.index.max()}")
print(f"   📊 Train: {X_train.shape[0]:,} samples | Test: {X_test.shape[0]:,} samples")
print(f"   🎯 Features: {X_train.shape[1]} | Target balance: {y.mean():.1%} bullish")

# ──────────────────────────────────────────────────────────────
# CUSTOM SCORER (Fβ WITH β = 0.5 for 2x precision weight)
# ──────────────────────────────────────────────────────────────
def precision_weighted_f1(y_true, y_pred):
    """F-beta score with beta=0.5 to weight precision 2x more than recall."""
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    beta = 0.5
    if p + r == 0:
        return 0.0
    return (1 + beta**2) * p * r / (beta**2 * p + r)

scorer = make_scorer(precision_weighted_f1, greater_is_better=True)

# ─────────────────────────────────────────────────────────────w─
# HYPERPARAMETER SEARCH
# ──────────────────────────────────────────────────────────────
param_dist = {
    "n_estimators":       [100, 150, 200, 250, 300, 400, 500],
    "max_depth":          [8, 10, 12, 15, 18, 20, None],
    "min_samples_split":  [5, 10, 15, 20, 25],
    "min_samples_leaf":   [2, 4, 6, 8, 10],
    "max_leaf_nodes":     [None, 50, 100, 200, 500],
    "max_features":       ["sqrt", "log2", 0.3, 0.5, 0.7],
    "bootstrap":          [True, False],
    "max_samples":        [0.7, 0.8, 0.9, 1.0],
    "class_weight":       [None, "balanced", "balanced_subsample"],
}

# Time-series cross-validation (respects temporal order)
cv = TimeSeriesSplit(n_splits=4)
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    scoring=scorer,
    n_iter=50,
    cv=cv,
    random_state=42,
    n_jobs=-1,
    verbose=1  # Show progress
)

print("\n🔍 Running hyperparameter optimization...")
start = time.time()
search.fit(X_train, y_train)
search_time = time.time() - start

print(f"⏱️  Optimization completed in {search_time:.1f}s")
print(f"🎯 Best CV score: {search.best_score_:.4f}")

# ──────────────────────────────────────────────────────────────
# RESULTS & EVALUATION
# ──────────────────────────────────────────────────────────────
print("\n🌟 OPTIMAL PARAMETERS:")
print("-" * 40)
for k, v in search.best_params_.items():
    print(f"   {k:<20}: {v}")

# Test set evaluation
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

print("\n📊 TEST SET PERFORMANCE:")
print("-" * 40)
print(f"   Accuracy                : {accuracy_score(y_test, y_pred):.4f}")
print(f"   Precision               : {precision_score(y_test, y_pred, zero_division=0):.4f}")
print(f"   Recall                  : {recall_score(y_test, y_pred, zero_division=0):.4f}")
print(f"   F1 (standard)           : {f1_score(y_test, y_pred, zero_division=0):.4f}")
print(f"   F1 (precision-weighted) : {precision_weighted_f1(y_test, y_pred):.4f}")
print(f"   ROC-AUC                 : {roc_auc_score(y_test, y_prob):.4f}")

# Feature importance (top 10)
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🌟 TOP 10 MOST IMPORTANT FEATURES:")
print("-" * 40)
for i, (_, row) in enumerate(feature_importance.head(10).iterrows(), 1):
    print(f"   {i:2d}. {row['feature']:<20}: {row['importance']:.4f}")

# Additional insights
print(f"\n📈 TRAINING INSIGHTS:")
print("-" * 40)
print(f"   Train period: {df.index[0]} to {df.index[split-1]}")
print(f"   Test period:  {df.index[split]} to {df.index[-1]}")
print(f"   CV folds:     {cv.n_splits}")
print(f"   Total params tested: {len(search.cv_results_['params'])}")

print(f"\n✅ Optimization complete! Use these parameters for your production Random Forest model.")

📊 Loading 4H Bitcoin data...
   📅 Date range: 2022-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Train: 5,672 samples | Test: 1,419 samples
   🎯 Features: 37 | Target balance: 50.6% bullish

🔍 Running hyperparameter optimization...
Fitting 4 folds for each of 50 candidates, totalling 200 fits
⏱️  Optimization completed in 147.0s
🎯 Best CV score: 0.5588

🌟 OPTIMAL PARAMETERS:
----------------------------------------
   n_estimators        : 300
   min_samples_split   : 15
   min_samples_leaf    : 4
   max_samples         : 0.9
   max_leaf_nodes      : 200
   max_features        : 0.3
   max_depth           : 10
   class_weight        : balanced
   bootstrap           : True

📊 TEST SET PERFORMANCE:
----------------------------------------
   Accuracy                : 0.5321
   Precision               : 0.5494
   Recall                  : 0.4890
   F1 (standard)           : 0.5174
   F1 (precision-weighted) : 0.5361
   ROC-AUC                 : 0.5344

🌟 TOP 10 MOST IMPORTANT FEATURES:
----

📊 Loading 4H Bitcoin data...
   📅 Date range: 2018-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Train: 12,684 samples | Test: 3,171 samples
   🎯 Features: 37 | Target balance: 51.1% bullish

🔍 Running hyperparameter optimization...
Fitting 4 folds for each of 50 candidates, totalling 200 fits
⏱️  Optimization completed in 343.2s
🎯 Best CV score: 0.5165

🌟 OPTIMAL PARAMETERS:
----------------------------------------
   n_estimators        : 400
   min_samples_split   : 5
   min_samples_leaf    : 10
   max_samples         : 0.9
   max_leaf_nodes      : 500
   max_features        : 0.5
   max_depth           : 8
   class_weight        : balanced_subsample
   bootstrap           : True

📊 TEST SET PERFORMANCE:
----------------------------------------
   Accuracy                : 0.5244
   Precision               : 0.5769
   Recall                  : 0.3351
   F1 (standard)           : 0.4240
   F1 (precision-weighted) : 0.5042
   ROC-AUC                 : 0.5501

🌟 TOP 10 MOST IMPORTANT FEATURES:
----------------------------------------
    1. roc_4h              : 0.0778
    2. buying_pressure     : 0.0613
    3. roc_24h             : 0.0469
    4. bb_position         : 0.0459
    5. fear_greed_score    : 0.0449
    6. atr_ratio           : 0.0431
    7. adx                 : 0.0415
    8. volume_mean_20      : 0.0404
    9. stoch_%K            : 0.0404
   10. CCI                 : 0.0386

📈 TRAINING INSIGHTS:
----------------------------------------
   Train period: 2018-01-01 00:00:00 to 2023-10-16 12:00:00
   Test period:  2023-10-16 16:00:00 to 2025-03-28 00:00:00
   CV folds:     4
   Total params tested: 50

✅ Optimization complete! Use these parameters for your production Random Forest model.

📊 Loading 4H Bitcoin data...
   📅 Date range: 2020-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Train: 9,180 samples | Test: 2,296 samples
   🎯 Features: 37 | Target balance: 51.0% bullish

🔍 Running hyperparameter optimization...
Fitting 4 folds for each of 50 candidates, totalling 200 fits
⏱️  Optimization completed in 314.8s
🎯 Best CV score: 0.5496

🌟 OPTIMAL PARAMETERS:
----------------------------------------
   n_estimators        : 300
   min_samples_split   : 15
   min_samples_leaf    : 2
   max_samples         : 0.9
   max_leaf_nodes      : 100
   max_features        : log2
   max_depth           : None
   class_weight        : None
   bootstrap           : True

📊 TEST SET PERFORMANCE:
----------------------------------------
   Accuracy                : 0.5366
   Precision               : 0.5727
   Recall                  : 0.4228
   F1 (standard)           : 0.4865
   F1 (precision-weighted) : 0.5348
   ROC-AUC                 : 0.5327

🌟 TOP 10 MOST IMPORTANT FEATURES:
----------------------------------------
    1. roc_4h              : 0.0611
    2. buying_pressure     : 0.0475
    3. bb_position         : 0.0431
    4. stoch_%K            : 0.0418
    5. fear_greed_score    : 0.0417
    6. price_vs_vwap       : 0.0397
    7. CCI                 : 0.0384
    8. roc_24h             : 0.0371
    9. stoch_%D            : 0.0367
   10. OBV                 : 0.0363

📈 TRAINING INSIGHTS:
----------------------------------------
   Train period: 2020-01-01 00:00:00 to 2024-03-10 08:00:00
   Test period:  2024-03-10 12:00:00 to 2025-03-28 00:00:00
   CV folds:     4
   Total params tested: 50

✅ Optimization complete! Use these parameters for your production Random Forest model.

📊 Loading 4H Bitcoin data...
   📅 Date range: 2022-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Train: 5,672 samples | Test: 1,419 samples
   🎯 Features: 37 | Target balance: 50.6% bullish

🔍 Running hyperparameter optimization...
Fitting 4 folds for each of 50 candidates, totalling 200 fits
⏱️  Optimization completed in 147.0s
🎯 Best CV score: 0.5588

🌟 OPTIMAL PARAMETERS:
----------------------------------------
   n_estimators        : 300
   min_samples_split   : 15
   min_samples_leaf    : 4
   max_samples         : 0.9
   max_leaf_nodes      : 200
   max_features        : 0.3
   max_depth           : 10
   class_weight        : balanced
   bootstrap           : True

📊 TEST SET PERFORMANCE:
----------------------------------------
   Accuracy                : 0.5321
   Precision               : 0.5494
   Recall                  : 0.4890
   F1 (standard)           : 0.5174
   F1 (precision-weighted) : 0.5361
   ROC-AUC                 : 0.5344

🌟 TOP 10 MOST IMPORTANT FEATURES:
----------------------------------------
    1. roc_4h              : 0.0645
    2. buying_pressure     : 0.0505
    3. volume_ratio        : 0.0435
    4. stoch_%K            : 0.0427
    5. bb_position         : 0.0419
    6. CCI                 : 0.0400
    7. atr_ratio           : 0.0394
    8. volume              : 0.0392
    9. fear_greed_score    : 0.0378
   10. stoch_%D            : 0.0377

📈 TRAINING INSIGHTS:
----------------------------------------
   Train period: 2022-01-01 00:00:00 to 2024-08-03 12:00:00
   Test period:  2024-08-03 16:00:00 to 2025-03-28 00:00:00
   CV folds:     4
   Total params tested: 50

✅ Optimization complete! Use these parameters for your production Random Forest model.

In [3]:
# =============================================================
#  RANDOM-FOREST  •  FINAL TRAINING WITH OPTIMAL PARAMS
# =============================================================
import numpy as np, pandas as pd, time, sys, warnings
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score)
warnings.filterwarnings("ignore")
np.random.seed(42)

# ──────────────────────────────────────────────────────────────
# 1) CONFIGURATION
# ──────────────────────────────────────────────────────────────
CSV_FILE     = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                    r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
TIME_COLUMN  = "timestamp"
TARGET_COL   = "target"
START_DATE   = "2018-01-01"
TEST_FRAC    = 0.20

DROP_COLS = [
    'open','high','low','high_low','high_close','low_close','typical_price',
    'volume_breakout','volume_breakdown','break_upper_band','break_lower_band',
    'vol_spike_1_5x','rsi_oversold','rsi_overbought','stoch_overbought',
    'stoch_oversold','cci_overbought','cci_oversold','near_upper_band',
    'near_lower_band','overbought_reversal','oversold_reversal',
    'ema_cross_up','ema_cross_down','macd_cross_up','macd_cross_down',
    'trending_market','trend_alignment','ema7_above_ema21','macd_rising',
    'bollinger_upper','bollinger_lower','bullish_scenario_1',
    'bullish_scenario_5','bearish_scenario_1'
]

best_params = {
    "n_estimators":     500,
    "max_depth":        6,
    "min_samples_split": 25,
    "min_samples_leaf": 8,
    "max_leaf_nodes":   200,
    "max_features":    "sqrt",
    "bootstrap":        True,
    "max_samples":      0.8,
    "class_weight":     None,
    "random_state":     42,
    "n_jobs":           -1
}

# ──────────────────────────────────────────────────────────────
# 2) LOAD & PREP DATA
# ──────────────────────────────────────────────────────────────
print("📊 Loading 4H Bitcoin data for final Random Forest training...")

if not CSV_FILE.exists():
    sys.exit(f"❌ File not found: {CSV_FILE}")

df = pd.read_csv(CSV_FILE, parse_dates=[TIME_COLUMN]).set_index(TIME_COLUMN).sort_index()
df = df.loc[START_DATE:].copy()

if TARGET_COL not in df.columns:
    sys.exit(f"❌ '{TARGET_COL}' column missing!")

X = df.drop(columns=[col for col in DROP_COLS if col in df.columns] + [TARGET_COL], errors="ignore")
y = df[TARGET_COL]

split = int(len(df) * (1 - TEST_FRAC))
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

print(f"   📅 Date range: {df.index.min()} to {df.index.max()}")
print(f"   📊 Train: {X_train.shape[0]:,} | Test: {X_test.shape[0]:,}")
print(f"   🎯 Features: {X_train.shape[1]} | Target balance: {y.mean():.1%} bullish")
print(f"   ⏰ Train period: {df.index[0]} to {df.index[split-1]}")
print(f"   🧪 Test period:  {df.index[split]} to {df.index[-1]}")

# ──────────────────────────────────────────────────────────────
# 3) TRAIN FINAL MODEL
# ──────────────────────────────────────────────────────────────
print(f"\n🚀 Training final Random Forest with optimal parameters...")
print("   Parameters:")
for k, v in best_params.items():
    print(f"      {k:<18}: {v}")

t0 = time.time()
rf_final = RandomForestClassifier(**best_params)
rf_final.fit(X_train, y_train)
training_time = time.time() - t0

print(f"🟢 Model trained successfully in {training_time:.1f}s")

# ──────────────────────────────────────────────────────────────
# 4) EVALUATE MODEL
# ──────────────────────────────────────────────────────────────
print(f"\n📊 FINAL MODEL EVALUATION")
print("=" * 40)

y_pred = rf_final.predict(X_test)
y_prob = rf_final.predict_proba(X_test)[:, 1] if rf_final.n_classes_ == 2 else rf_final.predict_proba(X_test).max(axis=1)

def precision_weighted_f1(y_true, y_pred):
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    beta = 0.5
    return (1 + beta**2) * p * r / (beta**2 * p + r) if (p + r) > 0 else 0.0

metrics = {
    "Accuracy":                 accuracy_score(y_test, y_pred),
    "Precision":                precision_score(y_test, y_pred, zero_division=0),
    "Recall":                   recall_score(y_test, y_pred, zero_division=0),
    "F1 (standard)":            f1_score(y_test, y_pred, zero_division=0),
    "F1 (precision-weighted)":  precision_weighted_f1(y_test, y_pred),
    "ROC-AUC":                  roc_auc_score(y_test, y_prob)
}

print("🎯 Test Set Performance:")
for k, v in metrics.items():
    print(f"   {k:<25}: {v:.4f}")

# ──────────────────────────────────────────────────────────────
# 5) FEATURE IMPORTANCE
# ──────────────────────────────────────────────────────────────
print(f"\n🌟 FEATURE IMPORTANCE ANALYSIS")
print("-" * 40)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_final.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 15 Most Important Features:")
for i, (_, row) in enumerate(feature_importance.head(15).iterrows(), 1):
    print(f"   {i:2d}. {row['feature']:<20}: {row['importance']:.4f}")

# ──────────────────────────────────────────────────────────────
# 6) SUMMARY
# ──────────────────────────────────────────────────────────────
print(f"\n🎉 TRAINING COMPLETE!")
print("=" * 50)
print(f"🎯 Model Performance Summary:")
print(f"   • Accuracy: {metrics['Accuracy']:.3f}")
print(f"   • Precision: {metrics['Precision']:.3f} (optimized metric)")
print(f"   • F1-weighted: {metrics['F1 (precision-weighted)']:.3f}")
print(f"   • Training time: {training_time:.1f}s")
print(f"   • Features used: {len(X_train.columns)}")
print(f"\n🚀 Ready for downstream use or ensemble integration!")


📊 Loading 4H Bitcoin data for final Random Forest training...
   📅 Date range: 2018-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Train: 12,684 | Test: 3,171
   🎯 Features: 37 | Target balance: 51.1% bullish
   ⏰ Train period: 2018-01-01 00:00:00 to 2023-10-16 12:00:00
   🧪 Test period:  2023-10-16 16:00:00 to 2025-03-28 00:00:00

🚀 Training final Random Forest with optimal parameters...
   Parameters:
      n_estimators      : 500
      max_depth         : 6
      min_samples_split : 25
      min_samples_leaf  : 8
      max_leaf_nodes    : 200
      max_features      : sqrt
      bootstrap         : True
      max_samples       : 0.8
      class_weight      : None
      random_state      : 42
      n_jobs            : -1
🟢 Model trained successfully in 1.3s

📊 FINAL MODEL EVALUATION
🎯 Test Set Performance:
   Accuracy                 : 0.5282
   Precision                : 0.5813
   Recall                   : 0.3454
   F1 (standard)            : 0.4333
   F1 (precision-weighted)  : 0.5114

📊 Loading 4H Bitcoin data for final Random Forest training...
   📅 Date range: 2016-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Train: 16,184 | Test: 4,046
   🎯 Features: 37 | Target balance: 51.8% bullish
   ⏰ Train period: 2016-01-01 00:00:00 to 2023-05-23 16:00:00
   🧪 Test period:  2023-05-23 20:00:00 to 2025-03-28 00:00:00

🚀 Training final Random Forest with optimal parameters...
   Parameters:
      n_estimators      : 300
      max_depth         : 15
      min_samples_split : 10
      min_samples_leaf  : 4
      max_leaf_nodes    : 200
      max_features      : sqrt
      bootstrap         : True
      max_samples       : 0.8
      class_weight      : balanced_subsample
      random_state      : 42
      n_jobs            : -1
🟢 Model trained successfully in 1.7s

📊 FINAL MODEL EVALUATION
========================================
🎯 Test Set Performance:
   Accuracy                 : 0.5309
   Precision                : 0.5776
   Recall                   : 0.3224
   F1 (standard)            : 0.4138
   F1 (precision-weighted)  : 0.4987
   ROC-AUC                  : 0.5489

🌟 FEATURE IMPORTANCE ANALYSIS
----------------------------------------
Top 15 Most Important Features:
    1. roc_4h              : 0.0530
    2. buying_pressure     : 0.0427
    3. bb_position         : 0.0406
    4. stoch_%K            : 0.0395
    5. fear_greed_score    : 0.0393
    6. atr_ratio           : 0.0385
    7. roc_24h             : 0.0376
    8. volume_ratio        : 0.0374
    9. adx                 : 0.0373
   10. stoch_%D            : 0.0373
   11. volume              : 0.0369
   12. price_vs_vwap       : 0.0363
   13. CCI                 : 0.0363
   14. volume_mean_20      : 0.0360
   15. parkinson_vol       : 0.0351

🎉 TRAINING COMPLETE!
==================================================
🎯 Model Performance Summary:
   • Accuracy: 0.531
   • Precision: 0.578 (optimized metric)
   • F1-weighted: 0.499
   • Training time: 1.7s
   • Features used: 37

🚀 Ready for downstream use or ensemble integration!

📊 Loading 4H Bitcoin data for final Random Forest training...
   📅 Date range: 2016-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Train: 16,184 | Test: 4,046
   🎯 Features: 37 | Target balance: 51.8% bullish
   ⏰ Train period: 2016-01-01 00:00:00 to 2023-05-23 16:00:00
   🧪 Test period:  2023-05-23 20:00:00 to 2025-03-28 00:00:00

🚀 Training final Random Forest with optimal parameters...
   Parameters:
      n_estimators      : 300
      max_depth         : 10
      min_samples_split : 15
      min_samples_leaf  : 4
      max_leaf_nodes    : 200
      max_features      : 0.3
      bootstrap         : True
      max_samples       : 0.9
      class_weight      : balanced_subsample
      random_state      : 42
      n_jobs            : -1
🟢 Model trained successfully in 2.7s

📊 FINAL MODEL EVALUATION
========================================
🎯 Test Set Performance:
   Accuracy                 : 0.5314
   Precision                : 0.5750
   Recall                   : 0.3359
   F1 (standard)            : 0.4241
   F1 (precision-weighted)  : 0.5033
   ROC-AUC                  : 0.5537

🌟 FEATURE IMPORTANCE ANALYSIS
----------------------------------------
Top 15 Most Important Features:
    1. roc_4h              : 0.0628
    2. buying_pressure     : 0.0481
    3. stoch_%K            : 0.0426
    4. bb_position         : 0.0426
    5. fear_greed_score    : 0.0407
    6. stoch_%D            : 0.0400
    7. volume_mean_20      : 0.0397
    8. atr_ratio           : 0.0396
    9. volume              : 0.0391
   10. roc_24h             : 0.0388
   11. volume_ratio        : 0.0383
   12. adx                 : 0.0379
   13. price_vs_vwap       : 0.0373
   14. RSI                 : 0.0367
   15. CCI                 : 0.0364

🎉 TRAINING COMPLETE!
==================================================
🎯 Model Performance Summary:
   • Accuracy: 0.531
   • Precision: 0.575 (optimized metric)
   • F1-weighted: 0.503
   • Training time: 2.7s
   • Features used: 37

🚀 Ready for downstream use or ensemble integration!

📊 Loading 4H Bitcoin data for final Random Forest training...
   📅 Date range: 2018-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Train: 12,684 | Test: 3,171
   🎯 Features: 37 | Target balance: 51.1% bullish
   ⏰ Train period: 2018-01-01 00:00:00 to 2023-10-16 12:00:00
   🧪 Test period:  2023-10-16 16:00:00 to 2025-03-28 00:00:00

🚀 Training final Random Forest with optimal parameters...
   Parameters:
      n_estimators      : 300
      max_depth         : 15
      min_samples_split : 10
      min_samples_leaf  : 4
      max_leaf_nodes    : 200
      max_features      : sqrt
      bootstrap         : True
      max_samples       : 0.8
      class_weight      : balanced_subsample
      random_state      : 42
      n_jobs            : -1
🟢 Model trained successfully in 1.4s

📊 FINAL MODEL EVALUATION
========================================
🎯 Test Set Performance:
   Accuracy                 : 0.5251
   Precision                : 0.5908
   Recall                   : 0.2947
   F1 (standard)            : 0.3932
   F1 (precision-weighted)  : 0.4919
   ROC-AUC                  : 0.5537

🌟 FEATURE IMPORTANCE ANALYSIS
----------------------------------------
Top 15 Most Important Features:
    1. roc_4h              : 0.0526
    2. buying_pressure     : 0.0456
    3. fear_greed_score    : 0.0421
    4. bb_position         : 0.0415
    5. roc_24h             : 0.0403
    6. stoch_%K            : 0.0389
    7. CCI                 : 0.0382
    8. adx                 : 0.0374
    9. stoch_%D            : 0.0365
   10. volume_mean_20      : 0.0363
   11. volume              : 0.0356
   12. atr_ratio           : 0.0356
   13. price_vs_vwap       : 0.0355
   14. volume_ratio        : 0.0351
   15. MACD_histogram      : 0.0339

🎉 TRAINING COMPLETE!
==================================================
🎯 Model Performance Summary:
   • Accuracy: 0.525
   • Precision: 0.591 (optimized metric)
   • F1-weighted: 0.492
   • Training time: 1.4s
   • Features used: 37

🚀 Ready for downstream use or ensemble integration!

📊 Loading 4H Bitcoin data for final Random Forest training...
   📅 Date range: 2018-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Train: 12,684 | Test: 3,171
   🎯 Features: 37 | Target balance: 51.1% bullish
   ⏰ Train period: 2018-01-01 00:00:00 to 2023-10-16 12:00:00
   🧪 Test period:  2023-10-16 16:00:00 to 2025-03-28 00:00:00

🚀 Training final Random Forest with optimal parameters...
   Parameters:
      n_estimators      : 400
      max_depth         : 8
      min_samples_split : 5
      min_samples_leaf  : 10
      max_leaf_nodes    : 500
      max_features      : 0.5
      bootstrap         : True
      max_samples       : 0.9
      class_weight      : balanced_subsample
      random_state      : 42
      n_jobs            : -1
🟢 Model trained successfully in 3.6s

📊 FINAL MODEL EVALUATION
========================================
🎯 Test Set Performance:
   Accuracy                 : 0.5244
   Precision                : 0.5769
   Recall                   : 0.3351
   F1 (standard)            : 0.4240
   F1 (precision-weighted)  : 0.5042
   ROC-AUC                  : 0.5501

🌟 FEATURE IMPORTANCE ANALYSIS
----------------------------------------
Top 15 Most Important Features:
    1. roc_4h              : 0.0778
    2. buying_pressure     : 0.0613
    3. roc_24h             : 0.0469
    4. bb_position         : 0.0459
    5. fear_greed_score    : 0.0449
    6. atr_ratio           : 0.0431
    7. adx                 : 0.0415
    8. volume_mean_20      : 0.0404
    9. stoch_%K            : 0.0404
   10. CCI                 : 0.0386
   11. stoch_%D            : 0.0377
   12. price_vs_vwap       : 0.0369
   13. volume              : 0.0354
   14. bollinger_width     : 0.0354
   15. volume_ratio        : 0.0353

🎉 TRAINING COMPLETE!
==================================================
🎯 Model Performance Summary:
   • Accuracy: 0.524
   • Precision: 0.577 (optimized metric)
   • F1-weighted: 0.504
   • Training time: 3.6s
   • Features used: 37

🚀 Ready for downstream use or ensemble integration!

📊 Loading data and training model with your best parameters...
   📊 Train: 12,684 | Test: 3,171 | Features: 37
🚀 Training Random Forest...
✅ Model trained in 1.4s

🎯 THRESHOLD SENSITIVITY ANALYSIS
==========================================================================================
Threshold  Accuracy   Precision   Recall     F1         Predictions  % of Test  % Change  
------------------------------------------------------------------------------------------
0.3        0.523      0.523       0.992      0.685      3140         99.0      %    +0.0%
0.4        0.538      0.540       0.778      0.637      2386         75.2      %    +0.0%
0.5        0.525      0.591       0.295      0.393      826          26.0      %    +0.0%
0.6        0.479      0.556       0.015      0.029      45           1.4       %   -94.6%
0.7        0.478      0.000       0.000      0.000      0            0.0       %  -100.0%
0.8        0.478      0.000       0.000      0.000      0            0.0       %  -100.0%

🏆 BEST THRESHOLDS BY METRIC:
------------------------------------------------------------
   Best Accuracy:  0.4  (Acc: 0.538, Prec: 0.540, Rec: 0.778)
   Best Precision: 0.5  (Prec: 0.591, Rec: 0.295, F1: 0.393)
   Best Recall:    0.3  (Rec: 0.992, Prec: 0.523, F1: 0.685)
   Best F1:        0.3  (F1: 0.685, Prec: 0.523, Rec: 0.992)

💡 TRADING STRATEGY RECOMMENDATIONS:
------------------------------------------------------------
   🛡️  Conservative:  No threshold achieves 65%+ precision
   ⚖️  Balanced:     0.3  (Prec: 0.523, Rec: 0.992, 3140 signals)
   ⚡ Aggressive:   0.4  (Rec: 0.778, 2386 signals)

📊 SIGNAL VOLUME ANALYSIS:
------------------------------------------------------------
   Default (0.5):   826 signals (26.0% of test set)
   High Volume:     3,140 signals at 0.3 (+0% vs default)
   Selective:       0 signals at 0.8 (-100% vs default)

📈 PERFORMANCE RANGES ACROSS THRESHOLDS:
------------------------------------------------------------
   Accuracy:   0.478 - 0.538
   Precision:  0.000 - 0.591
   Recall:     0.000 - 0.992
   F1 Score:   0.000 - 0.685
   Signals:    0 - 3,140

🎯 FINAL RECOMMENDATION:
============================================================
   🏆 Use threshold: 0.3
   📊 Performance:   Accuracy=0.523, Precision=0.523, Recall=0.992, F1=0.685
   📈 Signals:       3,140 (99.0% of test set)
   💡 Reason:        F1 score improved +74.1%

✅ Threshold analysis complete! Use threshold 0.3 for optimal performance.

📊 Loading data and training model with your best parameters...
   📊 Train: 12,684 | Test: 3,171 | Features: 37
🚀 Training Random Forest...
✅ Model trained in 1.7s

🎯 THRESHOLD SENSITIVITY ANALYSIS
==========================================================================================
Threshold  Accuracy   Precision   Recall     F1         Predictions  % of Test  % Change  
------------------------------------------------------------------------------------------
0.3        0.522      0.522       1.000      0.686      3171         100.0     %    +0.0%
0.4        0.534      0.534       0.855      0.657      2653         83.7      %    +0.0%
0.5        0.524      0.580       0.321      0.413      915          28.9      %    +0.0%
0.6        0.479      0.750       0.004      0.007      8            0.3       %   -99.1%
0.7        0.478      0.000       0.000      0.000      0            0.0       %  -100.0%
0.8        0.478      0.000       0.000      0.000      0            0.0       %  -100.0%

🏆 BEST THRESHOLDS BY METRIC:
------------------------------------------------------------
   Best Accuracy:  0.4  (Acc: 0.534, Prec: 0.534, Rec: 0.855)
   Best Precision: 0.6  (Prec: 0.750, Rec: 0.004, F1: 0.007)
   Best Recall:    0.3  (Rec: 1.000, Prec: 0.522, F1: 0.686)
   Best F1:        0.3  (F1: 0.686, Prec: 0.522, Rec: 1.000)

💡 TRADING STRATEGY RECOMMENDATIONS:
------------------------------------------------------------
   🛡️  Conservative:  0.6  (Prec: 0.750, 8 signals)
   ⚖️  Balanced:     0.3  (Prec: 0.522, Rec: 1.000, 3171 signals)
   ⚡ Aggressive:   0.4  (Rec: 0.855, 2653 signals)

📊 SIGNAL VOLUME ANALYSIS:
------------------------------------------------------------
   Default (0.5):   915 signals (28.9% of test set)
   High Volume:     3,171 signals at 0.3 (+0% vs default)
   Selective:       0 signals at 0.8 (-100% vs default)

📈 PERFORMANCE RANGES ACROSS THRESHOLDS:
------------------------------------------------------------
   Accuracy:   0.478 - 0.534
   Precision:  0.000 - 0.750
   Recall:     0.000 - 1.000
   F1 Score:   0.000 - 0.686
   Signals:    0 - 3,171

🎯 FINAL RECOMMENDATION:
============================================================
   🏆 Use threshold: 0.3
   📊 Performance:   Accuracy=0.522, Precision=0.522, Recall=1.000, F1=0.686
   📈 Signals:       3,171 (100.0% of test set)
   💡 Reason:        F1 score improved +66.1%

✅ Threshold analysis complete! Use threshold 0.3 for optimal performance.

In [13]:
# =============================================================
#  THRESHOLD EVALUATION  •  TEST YOUR TRAINED MODEL
# =============================================================
import numpy as np, pandas as pd, time, sys, warnings
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score)
warnings.filterwarnings("ignore")
np.random.seed(42)

# ──────────────────────────────────────────────────────────────
# 1) CONFIGURATION - PUT YOUR BEST PARAMETERS HERE
# ──────────────────────────────────────────────────────────────
CSV_FILE     = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                    r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
TIME_COLUMN  = "timestamp"
TARGET_COL   = "target"
START_DATE   = "2018-01-01"
TEST_FRAC    = 0.20

DROP_COLS = [
    'open','high','low','high_low','high_close','low_close','typical_price',
    'volume_breakout','volume_breakdown','break_upper_band','break_lower_band',
    'vol_spike_1_5x','rsi_oversold','rsi_overbought','stoch_overbought',
    'stoch_oversold','cci_overbought','cci_oversold','near_upper_band',
    'near_lower_band','overbought_reversal','oversold_reversal',
    'ema_cross_up','ema_cross_down','macd_cross_up','macd_cross_down',
    'trending_market','trend_alignment','ema7_above_ema21','macd_rising',
    'bollinger_upper','bollinger_lower','bullish_scenario_1',
    'bullish_scenario_5','bearish_scenario_1'
]

# 🎯 PUT YOUR BEST PARAMETERS HERE (from hyperparameter search)
BEST_PARAMS = {
    "n_estimators":     300,
    "max_depth":        15,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "max_leaf_nodes":   200,
    "max_features":    "sqrt",
    "bootstrap":        True,
    "max_samples":      0.8,
    "class_weight":     "balanced_subsample",
    "random_state":     42,
    "n_jobs":           -1
}

# ──────────────────────────────────────────────────────────────
# 2) LOAD DATA & TRAIN MODEL
# ──────────────────────────────────────────────────────────────
print("📊 Loading data and training model with your best parameters...")

if not CSV_FILE.exists():
    sys.exit(f"❌ File not found: {CSV_FILE}")

df = pd.read_csv(CSV_FILE, parse_dates=[TIME_COLUMN]).set_index(TIME_COLUMN).sort_index()
df = df.loc[START_DATE:].copy()

if TARGET_COL not in df.columns:
    sys.exit(f"❌ '{TARGET_COL}' column missing!")

X = df.drop(columns=[col for col in DROP_COLS if col in df.columns] + [TARGET_COL], errors="ignore")
y = df[TARGET_COL]

split = int(len(df) * (1 - TEST_FRAC))
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

print(f"   📊 Train: {X_train.shape[0]:,} | Test: {X_test.shape[0]:,} | Features: {X_train.shape[1]}")

# Train model
print("🚀 Training Random Forest...")
start_time = time.time()
model = RandomForestClassifier(**BEST_PARAMS)
model.fit(X_train, y_train)
train_time = time.time() - start_time

# Get predictions and probabilities
y_prob = model.predict_proba(X_test)[:, 1]
print(f"✅ Model trained in {train_time:.1f}s")

# ──────────────────────────────────────────────────────────────
# 3) THRESHOLD ANALYSIS
# ──────────────────────────────────────────────────────────────
print(f"\n🎯 THRESHOLD SENSITIVITY ANALYSIS")
print("=" * 90)
print(f"{'Threshold':<10} {'Accuracy':<10} {'Precision':<11} {'Recall':<10} {'F1':<10} {'Predictions':<12} {'% of Test':<10} {'% Change':<10}")
print("-" * 90)

thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
baseline_predictions = None
threshold_results = []

for i, threshold in enumerate(thresholds):
    # Apply threshold to probabilities
    y_pred_thresh = (y_prob >= threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_thresh)
    precision = precision_score(y_test, y_pred_thresh, zero_division=0)
    recall = recall_score(y_test, y_pred_thresh, zero_division=0)
    f1 = f1_score(y_test, y_pred_thresh, zero_division=0)
    
    # Count predictions
    positive_predictions = sum(y_pred_thresh)
    pct_of_test = (positive_predictions / len(y_test)) * 100
    
    # Set baseline (0.5 threshold) for comparison
    if threshold == 0.5:
        baseline_predictions = positive_predictions
    
    # Calculate percentage change from baseline
    if baseline_predictions is not None:
        pct_change = ((positive_predictions - baseline_predictions) / baseline_predictions * 100) if baseline_predictions > 0 else 0
    else:
        pct_change = 0
    
    threshold_results.append({
        'threshold': threshold,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'predictions': positive_predictions,
        'pct_of_test': pct_of_test,
        'pct_change': pct_change
    })
    
    # Display row
    print(f"{threshold:<10.1f} {accuracy:<10.3f} {precision:<11.3f} {recall:<10.3f} {f1:<10.3f} "
          f"{positive_predictions:<12} {pct_of_test:<10.1f}% {pct_change:>+7.1f}%")

# ──────────────────────────────────────────────────────────────
# 4) ANALYSIS & RECOMMENDATIONS
# ──────────────────────────────────────────────────────────────

# Find best thresholds for different objectives
best_accuracy = max(threshold_results, key=lambda x: x['accuracy'])
best_precision = max(threshold_results, key=lambda x: x['precision'])
best_recall = max(threshold_results, key=lambda x: x['recall'])
best_f1 = max(threshold_results, key=lambda x: x['f1'])

print(f"\n🏆 BEST THRESHOLDS BY METRIC:")
print("-" * 60)
print(f"   Best Accuracy:  {best_accuracy['threshold']:.1f}  "
      f"(Acc: {best_accuracy['accuracy']:.3f}, Prec: {best_accuracy['precision']:.3f}, Rec: {best_accuracy['recall']:.3f})")
print(f"   Best Precision: {best_precision['threshold']:.1f}  "
      f"(Prec: {best_precision['precision']:.3f}, Rec: {best_precision['recall']:.3f}, F1: {best_precision['f1']:.3f})")
print(f"   Best Recall:    {best_recall['threshold']:.1f}  "
      f"(Rec: {best_recall['recall']:.3f}, Prec: {best_recall['precision']:.3f}, F1: {best_recall['f1']:.3f})")
print(f"   Best F1:        {best_f1['threshold']:.1f}  "
      f"(F1: {best_f1['f1']:.3f}, Prec: {best_f1['precision']:.3f}, Rec: {best_f1['recall']:.3f})")

# Find balanced options
print(f"\n💡 TRADING STRATEGY RECOMMENDATIONS:")
print("-" * 60)

# Conservative (high precision, low false positives)
conservative = [r for r in threshold_results if r['precision'] >= 0.65]
if conservative:
    best_conservative = max(conservative, key=lambda x: x['recall'])
    print(f"   🛡️  Conservative:  {best_conservative['threshold']:.1f}  "
          f"(Prec: {best_conservative['precision']:.3f}, {best_conservative['predictions']} signals)")
else:
    print(f"   🛡️  Conservative:  No threshold achieves 65%+ precision")

# Balanced (good precision AND recall)
balanced = [r for r in threshold_results if r['precision'] >= 0.50 and r['recall'] >= 0.40]
if balanced:
    best_balanced = max(balanced, key=lambda x: x['f1'])
    print(f"   ⚖️  Balanced:     {best_balanced['threshold']:.1f}  "
          f"(Prec: {best_balanced['precision']:.3f}, Rec: {best_balanced['recall']:.3f}, {best_balanced['predictions']} signals)")
else:
    print(f"   ⚖️  Balanced:     No threshold achieves 50%+ precision AND 40%+ recall")

# Aggressive (high recall, catch more opportunities)
aggressive = [r for r in threshold_results if r['recall'] >= 0.50]
if aggressive:
    best_aggressive = max(aggressive, key=lambda x: x['precision'])
    print(f"   ⚡ Aggressive:   {best_aggressive['threshold']:.1f}  "
          f"(Rec: {best_aggressive['recall']:.3f}, {best_aggressive['predictions']} signals)")
else:
    print(f"   ⚡ Aggressive:   No threshold achieves 50%+ recall")

# Volume analysis
print(f"\n📊 SIGNAL VOLUME ANALYSIS:")
print("-" * 60)
baseline_result = next(r for r in threshold_results if r['threshold'] == 0.5)
print(f"   Default (0.5):   {baseline_result['predictions']:,} signals ({baseline_result['pct_of_test']:.1f}% of test set)")

high_volume = [r for r in threshold_results if r['predictions'] >= baseline_result['predictions'] * 1.5]
if high_volume:
    best_volume = min(high_volume, key=lambda x: x['threshold'])  # Lowest threshold with high volume
    print(f"   High Volume:     {best_volume['predictions']:,} signals at {best_volume['threshold']:.1f} "
          f"({best_volume['pct_change']:+.0f}% vs default)")

low_volume = [r for r in threshold_results if r['predictions'] <= baseline_result['predictions'] * 0.6]
if low_volume:
    best_selective = max(low_volume, key=lambda x: x['threshold'])  # Highest threshold with low volume
    print(f"   Selective:       {best_selective['predictions']:,} signals at {best_selective['threshold']:.1f} "
          f"({best_selective['pct_change']:+.0f}% vs default)")

# Performance ranges
print(f"\n📈 PERFORMANCE RANGES ACROSS THRESHOLDS:")
print("-" * 60)
print(f"   Accuracy:   {min(r['accuracy'] for r in threshold_results):.3f} - {max(r['accuracy'] for r in threshold_results):.3f}")
print(f"   Precision:  {min(r['precision'] for r in threshold_results):.3f} - {max(r['precision'] for r in threshold_results):.3f}")
print(f"   Recall:     {min(r['recall'] for r in threshold_results):.3f} - {max(r['recall'] for r in threshold_results):.3f}")
print(f"   F1 Score:   {min(r['f1'] for r in threshold_results):.3f} - {max(r['f1'] for r in threshold_results):.3f}")
print(f"   Signals:    {min(r['predictions'] for r in threshold_results):,} - {max(r['predictions'] for r in threshold_results):,}")

# ──────────────────────────────────────────────────────────────
# 5) DATE-BY-DATE PREDICTIONS OUTPUT
# ──────────────────────────────────────────────────────────────
print(f"\n📅 GENERATING DATE-BY-DATE PREDICTIONS...")

# Use default 0.5 threshold for predictions
y_pred_final = (y_prob >= 0.5).astype(int)

# Create detailed predictions DataFrame
predictions_df = pd.DataFrame({
    'timestamp': X_test.index,
    'actual': y_test.values,
    'probability': y_prob,
    'predicted': y_pred_final
})

# Add prediction confidence categories
predictions_df['confidence'] = pd.cut(
    predictions_df['probability'], 
    bins=[0, 0.3, 0.4, 0.6, 0.7, 1.0],
    labels=['Very_Low', 'Low', 'Medium', 'High', 'Very_High']
)

# Add correctness
predictions_df['correct'] = (predictions_df['actual'] == predictions_df['predicted'])

print(f"📊 SAMPLE PREDICTIONS (First 20 rows):")
print("=" * 90)
print(f"{'Date':<20} {'Actual':<7} {'Predicted':<10} {'Probability':<12} {'Confidence':<12} {'Correct':<8}")
print("-" * 90)

for i, (_, row) in enumerate(predictions_df.head(20).iterrows()):
    date_str = row['timestamp'].strftime('%Y-%m-%d %H:%M')
    actual_str = "🟢 Bull" if row['actual'] == 1 else "🔴 Bear"
    pred_str = "🟢 Bull" if row['predicted'] == 1 else "🔴 Bear"
    prob_str = f"{row['probability']:.4f}"
    conf_str = str(row['confidence'])
    correct_str = "✅" if row['correct'] else "❌"
    
    print(f"{date_str:<20} {actual_str:<7} {pred_str:<10} {prob_str:<12} {conf_str:<12} {correct_str:<8}")

# Show statistics by confidence level
print(f"\n📈 ACCURACY BY CONFIDENCE LEVEL:")
print("-" * 50)
confidence_stats = predictions_df.groupby('confidence').agg({
    'correct': ['count', 'sum', 'mean'],
    'probability': ['mean', 'std']
}).round(4)

for conf_level in predictions_df['confidence'].cat.categories:
    if conf_level in confidence_stats.index:
        stats = confidence_stats.loc[conf_level]
        count = int(stats[('correct', 'count')])
        accuracy = stats[('correct', 'mean')]
        avg_prob = stats[('probability', 'mean')]
        
        print(f"   {conf_level:<12}: {count:>4} predictions, {accuracy:.1%} accuracy, avg prob: {avg_prob:.3f}")

# Save predictions to CSV for ensemble analysis
output_file = CSV_FILE.parent / f"rf_predictions_{START_DATE.replace('-', '')}.csv"

# Also save to Desktop for easy access
desktop_path = Path.home() / "Desktop"
desktop_file = desktop_path / f"bitcoin_rf_predictions_{START_DATE.replace('-', '')}.csv"

try:
    predictions_df.to_csv(desktop_file, index=False)
    desktop_saved = True
except:
    desktop_saved = False

print(f"\n💾 PREDICTIONS SAVED:")
print(f"   Primary file: {output_file}")
if desktop_saved:
    print(f"   Desktop copy: {desktop_file}")
    print(f"   ✅ Ready to download from Desktop!")
else:
    print(f"   ❌ Could not save to Desktop, check permissions")
print(f"   Rows: {len(predictions_df):,}")
print(f"   Columns: {list(predictions_df.columns)}")

# Create a summary file for quick reference
summary_data = {
    'Model': ['Random_Forest'],
    'Start_Date': [START_DATE],
    'Test_Samples': [len(predictions_df)],
    'Accuracy': [predictions_df['correct'].mean()],
    'Precision': [precision_score(predictions_df['actual'], predictions_df['predicted'])],
    'Recall': [recall_score(predictions_df['actual'], predictions_df['predicted'])],
    'F1_Score': [f1_score(predictions_df['actual'], predictions_df['predicted'])],
    'Avg_Probability': [predictions_df['probability'].mean()],
    'Bull_Signals': [sum(predictions_df['predicted'])],
    'Bull_Percentage': [sum(predictions_df['predicted'])/len(predictions_df)*100],
    'File_Path': [str(desktop_file if desktop_saved else output_file)]
}

summary_df = pd.DataFrame(summary_data)
summary_file = desktop_path / f"bitcoin_model_summary_{START_DATE.replace('-', '')}.csv" if desktop_saved else CSV_FILE.parent / f"model_summary_{START_DATE.replace('-', '')}.csv"

try:
    summary_df.to_csv(summary_file, index=False)
    print(f"   Summary file: {summary_file}")
except:
    print(f"   ❌ Could not save summary file")

# Summary for ensemble integration
print(f"\n🤖 ENSEMBLE INTEGRATION READY:")
print("-" * 50)
print(f"   Model Type:       Random Forest")
print(f"   Test Period:      {predictions_df['timestamp'].min()} to {predictions_df['timestamp'].max()}")
print(f"   Total Predictions: {len(predictions_df):,}")
print(f"   Bullish Signals:   {sum(predictions_df['predicted']):,} ({sum(predictions_df['predicted'])/len(predictions_df)*100:.1f}%)")
print(f"   Overall Accuracy:  {predictions_df['correct'].mean():.1%}")
print(f"   Avg Probability:   {predictions_df['probability'].mean():.3f}")

# Show high confidence predictions (for ensemble voting)
high_conf_mask = predictions_df['probability'] >= 0.7
low_conf_mask = predictions_df['probability'] <= 0.3

print(f"\n🎯 PROBABILITY DISTRIBUTION:")
print("-" * 50)
if high_conf_mask.any():
    high_conf_bull = high_conf_mask & (predictions_df['predicted'] == 1)
    print(f"   High confidence Bull: {sum(high_conf_bull):,} (prob ≥ 0.7)")
    if sum(high_conf_bull) > 0:
        print(f"   Sample dates: {', '.join(predictions_df[high_conf_bull]['timestamp'].dt.strftime('%Y-%m-%d').head(3).tolist())}")

if low_conf_mask.any():
    high_conf_bear = low_conf_mask & (predictions_df['predicted'] == 0)
    print(f"   High confidence Bear: {sum(high_conf_bear):,} (prob ≤ 0.3)")
    if sum(high_conf_bear) > 0:
        print(f"   Sample dates: {', '.join(predictions_df[high_conf_bear]['timestamp'].dt.strftime('%Y-%m-%d').head(3).tolist())}")

print(f"   💡 You can experiment with any threshold using the 'probability' column!")

# ──────────────────────────────────────────────────────────────
# 6) FINAL RECOMMENDATION
# ──────────────────────────────────────────────────────────────
print(f"\n🎯 FINAL RECOMMENDATION:")
print("=" * 60)

# Choose best overall threshold
if best_f1['f1'] > baseline_result['f1'] * 1.1:  # If F1 improved by 10%+
    recommended = best_f1
    reason = f"F1 score improved {((best_f1['f1'] / baseline_result['f1']) - 1) * 100:+.1f}%"
elif best_recall['recall'] > baseline_result['recall'] * 1.5:  # If recall improved significantly
    recommended = best_recall
    reason = f"Recall improved {((best_recall['recall'] / baseline_result['recall']) - 1) * 100:+.1f}%"
else:
    recommended = baseline_result
    reason = "Default threshold performs best"

print(f"   🏆 Use threshold: {recommended['threshold']:.1f}")
print(f"   📊 Performance:   Accuracy={recommended['accuracy']:.3f}, Precision={recommended['precision']:.3f}, "
      f"Recall={recommended['recall']:.3f}, F1={recommended['f1']:.3f}")
print(f"   📈 Signals:       {recommended['predictions']:,} ({recommended['pct_of_test']:.1f}% of test set)")
print(f"   💡 Reason:        {reason}")

print(f"\n✅ Threshold analysis complete! Use threshold {recommended['threshold']:.1f} for optimal performance.")

📊 Loading data and training model with your best parameters...
   📊 Train: 12,684 | Test: 3,171 | Features: 37
🚀 Training Random Forest...
✅ Model trained in 1.4s

🎯 THRESHOLD SENSITIVITY ANALYSIS
Threshold  Accuracy   Precision   Recall     F1         Predictions  % of Test  % Change  
------------------------------------------------------------------------------------------
0.3        0.523      0.523       0.992      0.685      3140         99.0      %    +0.0%
0.4        0.538      0.540       0.778      0.637      2386         75.2      %    +0.0%
0.5        0.525      0.591       0.295      0.393      826          26.0      %    +0.0%
0.6        0.479      0.556       0.015      0.029      45           1.4       %   -94.6%
0.7        0.478      0.000       0.000      0.000      0            0.0       %  -100.0%
0.8        0.478      0.000       0.000      0.000      0            0.0       %  -100.0%

🏆 BEST THRESHOLDS BY METRIC:
---------------------------------------------------

In [6]:
"""
Clean Random-Forest optimisation WITH per-trial prints & CSV log
===============================================================

Searches 100 random hyper-parameter combinations, prints metrics for every
trial and stores all trial details for later analysis.
"""

import numpy as np, pandas as pd, time, sys, warnings, joblib, json
from pathlib import Path
from datetime import datetime
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterSampler, TimeSeriesSplit
from sklearn.metrics import (
    precision_score, recall_score, f1_score, accuracy_score, roc_auc_score,
    confusion_matrix, classification_report
)

warnings.filterwarnings("ignore")
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# ═════════════════════════════ CONFIG ════════════════════════════════
CSV_FILE   = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                  r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
TIME_COL   = "timestamp"
TARGET_COL = "target"
START_DATE = "2018-01-01"
TEST_FRAC  = 0.20

N_ITER          = 100        # trials
CV_SPLITS       = 5          # TimeSeriesSplit folds
BETA            = 0.5        # F-beta → 2× precision weight
TRIAL_CSV_TMPL  = "rf_trial_details_{}.csv"
MODEL_OUT       = "rf_optimized_model.joblib"
RESULTS_JSON_TMPL = "rf_optimization_results_{}.json"

# Same comprehensive drop list you used before
DROP_COLS = [
    # … (unchanged – keep the full list you had)
    'open','high','low','close','high_low','high_close','low_close', 'typical_price',
    'vwap_24h','close_4h','volume_breakout','volume_breakdown','break_upper_band',
    'break_lower_band','vol_spike_1_5x','rsi_oversold','rsi_overbought',
    'stoch_overbought','stoch_oversold','cci_overbought','cci_oversold',
    'near_upper_band','near_lower_band','overbought_reversal','oversold_reversal',
    'ema_cross_up','ema_cross_down','macd_cross_up','macd_cross_down',
    'trending_market','trend_alignment','ema7_above_ema21','macd_rising',
    'bollinger_upper','bollinger_lower','bollinger_width',
    'resistance_level','support_level',
    'bullish_scenario_1','bullish_scenario_2','bullish_scenario_3',
    'bullish_scenario_4','bullish_scenario_5','bullish_scenario_6',
    'bearish_scenario_1','bearish_scenario_2','bearish_scenario_3',
    'bearish_scenario_4','bearish_scenario_6',
    'EMA_7','EMA_21','SMA_20','SMA_50','MACD_line','MACD_signal',
    'timestamp','date','Unnamed: 0'
]

# ═══════════════════════ HELPERS ═════════════════════════════════════
def f_beta(y_true, y_pred, beta=BETA):
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score   (y_true, y_pred, zero_division=0)
    if p + r == 0:
        return 0.0
    return (1 + beta**2) * p * r / (beta**2 * p + r)

# ═════════════════════ DATA LOAD ═════════════════════════════════════
print("🚀 Random-Forest optimisation – per-trial logging")
if not CSV_FILE.exists():
    sys.exit(f"❌ File not found: {CSV_FILE}")

df = (pd.read_csv(CSV_FILE, parse_dates=[TIME_COL])
        .set_index(TIME_COL)
        .sort_index()
        .loc[START_DATE:])

if TARGET_COL not in df.columns:
    sys.exit(f"❌ Target column '{TARGET_COL}' missing!")

X = df.drop(columns=[c for c in DROP_COLS if c in df.columns] + [TARGET_COL], errors="ignore")
y = df[TARGET_COL]

mask = ~(X.isnull().any(axis=1) | y.isnull())
X, y = X[mask], y[mask]
df = df[mask]                           # cleaned index for nice date prints

split_idx = int(len(X) * (1 - TEST_FRAC))
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"   Samples: {len(X):,}   Train/Val: {len(X_train):,}   Test: {len(X_test):,}")
print(f"   Features: {X.shape[1]}   Pos-rate train: {y_train.mean():.3f}")

# ═══════════════ PARAMETER SPACE ══════════════════════════════════════
param_dist = {
    "n_estimators"        : randint(100, 800),
    "max_depth"           : [8,10,12,15,18,20,25,None],
    "min_samples_split"   : randint(10, 50),
    "min_samples_leaf"    : randint(3, 15),
    "max_leaf_nodes"      : [None, 50,100,200,300,500,1000],
    "max_features"        : ["sqrt","log2",0.3,0.4,0.5,0.6,0.7,0.8],
    "bootstrap"           : [True],          # keep OOB enabled
    "class_weight"        : [None,"balanced","balanced_subsample",
                             {0:1,1:2},{0:1,1:3},{0:1,1:1.5}],
    "criterion"           : ["gini","entropy"],
    "min_impurity_decrease": uniform(0.0,0.01)
}

param_sampler = list(ParameterSampler(
    param_dist, n_iter=N_ITER, random_state=RANDOM_STATE))

cv = TimeSeriesSplit(n_splits=CV_SPLITS)

# ═══════════════ SEARCH LOOP ══════════════════════════════════════════
results = []
best_score  = -np.inf
best_params = None

print(f"\n🔍 {N_ITER} trials × {CV_SPLITS}-fold TimeSeriesSplit\n")

tic_all = time.time()
for i, params in enumerate(param_sampler, 1):
    rf = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1,
                                oob_score=True, **params)

    fold_prec, fold_rec, fold_f1, fold_fb = [], [], [], []
    for train_idx, val_idx in cv.split(X_train):
        rf.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
        preds = rf.predict(X_train.iloc[val_idx])

        fold_prec.append(precision_score(y_train.iloc[val_idx], preds, zero_division=0))
        fold_rec .append(recall_score   (y_train.iloc[val_idx], preds, zero_division=0))
        fold_f1  .append(f1_score       (y_train.iloc[val_idx], preds, zero_division=0))
        fold_fb  .append(f_beta         (y_train.iloc[val_idx], preds))

    trial_metrics = {
        "precision": np.mean(fold_prec),
        "recall"   : np.mean(fold_rec),
        "f1"       : np.mean(fold_f1),
        "f_beta"   : np.mean(fold_fb)
    }
    results.append({**params, **trial_metrics})

    # live print
    print(f"Trial {i:03d}/{N_ITER} | "
          f"P={trial_metrics['precision']:.3f} "
          f"R={trial_metrics['recall']:.3f} "
          f"F1={trial_metrics['f1']:.3f} "
          f"Fβ={trial_metrics['f_beta']:.3f} | "
          f"n_est={params['n_estimators']:<3d} "
          f"max_depth={params['max_depth']} "
          f"min_split={params['min_samples_split']:<2d} "
          f"min_leaf={params['min_samples_leaf']:<2d} "
          f"class_w={params['class_weight']}")

    if trial_metrics["f_beta"] > best_score:
        best_score, best_params = trial_metrics["f_beta"], params

toc_all = time.time()
print(f"\n⏱  Search finished in {(toc_all - tic_all)/60:.1f} min")
print(f"🏆 Best CV Fβ={best_score:.4f} with params: {best_params}")

# save all trials
ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
trial_csv = TRIAL_CSV_TMPL.format(ts)
pd.DataFrame(results).to_csv(trial_csv, index=False)
print(f"💾 Trial details saved → {trial_csv}")

# ═════════════ retrain best model & evaluate ══════════════════════════
best_model = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1,
                                    oob_score=True, **best_params)
best_model.fit(X_train, y_train)
y_pred  = best_model.predict(X_test)
y_prob  = best_model.predict_proba(X_test)[:,1]

precision = precision_score(y_test, y_pred, zero_division=0)
recall    = recall_score   (y_test, y_pred, zero_division=0)
f1        = f1_score       (y_test, y_pred, zero_division=0)
f_beta_ts = f_beta         (y_test, y_pred)
acc       = accuracy_score (y_test, y_pred)
auc       = roc_auc_score  (y_test, y_prob)

print("\n📊 TEST-WINDOW PERFORMANCE")
print("──────────────────────────")
print(f"Precision   : {precision:.4f}")
print(f"Recall      : {recall:.4f}")
print(f"F1          : {f1:.4f}")
print(f"Fβ (β=0.5)  : {f_beta_ts:.4f}")
print(f"Accuracy    : {acc:.4f}")
print(f"ROC-AUC     : {auc:.4f}")
print(f"Positives   : {y_pred.sum()} / {len(y_pred)}")

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=["Down","Up"]))

cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix: TN={cm[0,0]}  FP={cm[0,1]}  FN={cm[1,0]}  TP={cm[1,1]}")

# ═════════════ save artefacts ═════════════════════════════════════════
joblib.dump(best_model, MODEL_OUT)
print(f"\n💾 Best model saved → {MODEL_OUT}")

json.dump({
    "timestamp"    : ts + "Z",
    "best_params"  : best_params,
    "best_cv_f_beta": best_score,
    "test_metrics" : {
        "precision": precision, "recall": recall, "f1": f1,
        "f_beta": f_beta_ts, "accuracy": acc, "auc": auc
    }
}, open(RESULTS_JSON_TMPL.format(ts), "w"), indent=2)

print("\n🎉 DONE – you now have per-trial visibility and a tidy CSV of every run.")


🚀 Random-Forest optimisation – per-trial logging
   Samples: 15,855   Train/Val: 12,684   Test: 3,171
   Features: 26   Pos-rate train: 0.508

🔍 100 trials × 5-fold TimeSeriesSplit

Trial 001/100 | P=0.509 R=1.000 F1=0.674 Fβ=0.564 | n_est=314 max_depth=25 min_split=28 min_leaf=12 class_w={0: 1, 1: 2}
Trial 002/100 | P=0.559 R=0.453 F1=0.494 Fβ=0.528 | n_est=408 max_depth=None min_split=31 min_leaf=5  class_w=balanced_subsample
Trial 003/100 | P=0.545 R=0.499 F1=0.512 Fβ=0.529 | n_est=260 max_depth=15 min_split=30 min_leaf=14 class_w=balanced
Trial 004/100 | P=0.511 R=0.946 F1=0.662 Fβ=0.562 | n_est=799 max_depth=20 min_split=37 min_leaf=13 class_w={0: 1, 1: 2}
Trial 005/100 | P=0.511 R=0.936 F1=0.657 Fβ=0.561 | n_est=584 max_depth=25 min_split=12 min_leaf=6  class_w={0: 1, 1: 1.5}
Trial 006/100 | P=0.550 R=0.514 F1=0.524 Fβ=0.537 | n_est=341 max_depth=18 min_split=23 min_leaf=11 class_w=balanced_subsample
Trial 007/100 | P=0.531 R=0.722 F1=0.591 Fβ=0.550 | n_est=527 max_depth=10 min_s

In [ ]:
"""
Enhanced Random Forest Final Training & Prediction
=================================================
Improved version with better data preprocessing, comprehensive evaluation,
and proper CSV output format matching your other models.
"""

import numpy as np, pandas as pd, time, sys, warnings, joblib
from pathlib import Path
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, fbeta_score, roc_auc_score, 
                             confusion_matrix, classification_report)
import json

warnings.filterwarnings("ignore")
np.random.seed(42)

# ══════════════════════════════════════════════════════════════════════
# ENHANCED CONFIGURATION
# ══════════════════════════════════════════════════════════════════════
CSV_FILE = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
TIME_COLUMN = "timestamp"
TARGET_COL = "target"
START_DATE = "2018-01-01"
TEST_FRAC = 0.20
DECISION_THRESHOLD = 0.5
BETA_VALUE = 0.5  # For F-beta score (precision-weighted)

# Output files
MODEL_OUT = "rf_optimized_final.joblib"
SCALER_OUT = "rf_scaler_final.pkl"
PREDICTIONS_OUT = "rf_predictions.csv"
SUMMARY_JSON = "rf_training_summary.json"

# Enhanced DROP_COLS (comprehensive - same as optimization)
DROP_COLS = [
    'open', 'high', 'low', 'close',  # CRITICAL: No price data leakage
    'high_low', 'high_close', 'low_close', 'typical_price',
    'vwap_24h', 'close_4h',  # Additional price-derived features
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'rsi_oversold', 'rsi_overbought', 'stoch_overbought',
    'stoch_oversold', 'cci_overbought', 'cci_oversold', 'near_upper_band',
    'near_lower_band', 'overbought_reversal', 'oversold_reversal',
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'ema7_above_ema21', 'macd_rising',
    'bollinger_upper', 'bollinger_lower', 'bollinger_width',  # Price-based levels
    'resistance_level', 'support_level',  # Price levels
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',  # Moving averages with price info
    'MACD_line', 'MACD_signal',  # Price-derived indicators
    'timestamp', 'date', 'Unnamed: 0'  # Non-predictive columns
]

# OPTIMAL PARAMETERS (put your best parameters here)
BEST_PARAMS = {
    "n_estimators": 300,
    "max_depth": 15,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "max_leaf_nodes": 200,
    "max_features": "sqrt",
    "bootstrap": True,
    "class_weight": "balanced_subsample",
    "criterion": "gini",
    "min_impurity_decrease": 0.0,
    "random_state": 42,
    "n_jobs": -1,
    "oob_score": True  # For additional validation
}

# ══════════════════════════════════════════════════════════════════════
# HELPER FUNCTIONS
# ══════════════════════════════════════════════════════════════════════
def precision_weighted_f_beta(y_true, y_pred, beta=BETA_VALUE):
    """F-beta score with configurable beta for precision weighting."""
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    
    if p + r == 0:
        return 0.0
    
    return (1 + beta**2) * p * r / (beta**2 * p + r)

# ══════════════════════════════════════════════════════════════════════
# DATA LOADING AND PREPROCESSING
# ══════════════════════════════════════════════════════════════════════
print("🚀 Enhanced Random Forest Final Training")
print("=" * 50)
print("📊 Loading and preprocessing data...")

if not CSV_FILE.exists():
    sys.exit(f"❌ File not found: {CSV_FILE}")

# Load data
df = pd.read_csv(CSV_FILE, parse_dates=[TIME_COLUMN]).set_index(TIME_COLUMN).sort_index()
df = df.loc[START_DATE:].copy()

print(f"   📅 Date range: {df.index.min()} to {df.index.max()}")
print(f"   📊 Raw data shape: {df.shape}")

# Verify target column exists
if TARGET_COL not in df.columns:
    sys.exit(f"❌ Target column '{TARGET_COL}' not found!")

# Feature engineering and cleaning
X = df.drop(columns=[col for col in DROP_COLS if col in df.columns] + [TARGET_COL], errors="ignore")
y = df[TARGET_COL]

# Remove any remaining NaN values
initial_size = len(X)
mask = ~(X.isnull().any(axis=1) | y.isnull())
X, y = X[mask], y[mask]
df_clean = df[mask]

print(f"   🧹 Cleaned data: {len(X):,} samples ({initial_size - len(X)} removed)")
print(f"   🎯 Features: {X.shape[1]} | Target balance: {y.mean():.1%} bullish")

# Validate no price leakage
price_columns = ['open', 'high', 'low', 'close', 'price']
found_price_cols = [col for col in X.columns if any(price_word in col.lower() for price_word in price_columns)]
if found_price_cols:
    print(f"⚠️  Warning: Potential price leakage detected: {found_price_cols}")

print(f"   ✅ Features used: {list(X.columns)}")

# Chronological split (CRITICAL: maintains time order)
split = int(len(X) * (1 - TEST_FRAC))
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

print(f"\n📈 Data Split:")
print(f"   Train: {X_train.shape[0]:,} samples ({df_clean.index[0]} to {df_clean.index[split-1]})")
print(f"   Test:  {X_test.shape[0]:,} samples ({df_clean.index[split]} to {df_clean.index[-1]})")
print(f"   Train positive rate: {y_train.mean():.3f}")
print(f"   Test positive rate: {y_test.mean():.3f}")

# Optional: Feature scaling (Random Forest doesn't require it, but can help with consistency)
print(f"\n🔄 Feature scaling (optional for RF)...")
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save scaler for consistency with other models
joblib.dump(scaler, SCALER_OUT)
print(f"   Scaler saved: {SCALER_OUT}")

# ══════════════════════════════════════════════════════════════════════
# MODEL TRAINING
# ══════════════════════════════════════════════════════════════════════
print(f"\n🏗️ Building Random Forest with optimal parameters...")
print(f"   Estimators: {BEST_PARAMS['n_estimators']}")
print(f"   Max depth: {BEST_PARAMS['max_depth']}")
print(f"   Class weight: {BEST_PARAMS['class_weight']}")
print(f"   Max features: {BEST_PARAMS['max_features']}")

print(f"\n🚀 Training Random Forest...")
start_time = time.time()

# Use original features (RF handles scaling internally)
model = RandomForestClassifier(**BEST_PARAMS)
model.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"✅ Model trained in {train_time:.1f}s")

# ══════════════════════════════════════════════════════════════════════
# MODEL EVALUATION
# ══════════════════════════════════════════════════════════════════════
print(f"\n📊 Evaluating model performance...")

# Predictions
y_prob = model.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= DECISION_THRESHOLD).astype(int)

# Calculate comprehensive metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
f_beta = precision_weighted_f_beta(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print(f"\n🎯 COMPREHENSIVE PERFORMANCE (Threshold = {DECISION_THRESHOLD})")
print("=" * 60)
print(f"   Accuracy                : {accuracy:.4f}")
print(f"   Precision               : {precision:.4f} ⭐")
print(f"   Recall                  : {recall:.4f}")
print(f"   F1 Score                : {f1:.4f}")
print(f"   F-beta (β={BETA_VALUE})         : {f_beta:.4f} 🎯")
print(f"   ROC AUC                 : {auc:.4f}")
print(f"   Positive predictions    : {np.sum(y_pred)} / {len(y_pred)}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(f"\n📋 Confusion Matrix:")
print(f"   True Negatives (TN): {cm[0,0]}")
print(f"   False Positives (FP): {cm[0,1]}")
print(f"   False Negatives (FN): {cm[1,0]}")
print(f"   True Positives (TP): {cm[1,1]}")

# Classification report
print(f"\n📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Down", "Up"]))

# OOB Score (if available)
if hasattr(model, 'oob_score_'):
    print(f"\n🎯 Out-of-Bag Score: {model.oob_score_:.4f}")

# ══════════════════════════════════════════════════════════════════════
# FEATURE IMPORTANCE ANALYSIS
# ══════════════════════════════════════════════════════════════════════
print(f"\n🌟 Feature Importance Analysis...")

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🌟 TOP 10 MOST IMPORTANT FEATURES:")
print("=" * 50)
for i, (_, row) in enumerate(feature_importance.head(10).iterrows(), 1):
    print(f"   {i:2d}. {row['feature']:<25}: {row['importance']:.4f}")

# Feature importance insights
top_5_importance = feature_importance.head(5)['importance'].sum()
top_10_importance = feature_importance.head(10)['importance'].sum()

print(f"\n📈 Feature Importance Insights:")
print(f"   Top 5 features explain:  {top_5_importance:.1%} of decisions")
print(f"   Top 10 features explain: {top_10_importance:.1%} of decisions")

# ══════════════════════════════════════════════════════════════════════
# GENERATE PREDICTIONS CSV (MATCHING YOUR OTHER MODELS)
# ══════════════════════════════════════════════════════════════════════
print(f"\n📁 Generating predictions CSV...")

# Create predictions dataframe matching your other models' format
prob_up = y_prob
prob_down = 1.0 - prob_up
winning_prob = np.maximum(prob_up, prob_down)

predictions_df = pd.DataFrame({
    'timestamp': X_test.index.strftime('%Y-%m-%d %H:%M:%S'),
    'prob_up': prob_up,
    'prob_down': prob_down,
    'winning_prob': winning_prob,
    'prediction': y_pred,
    'actual': y_test.values
})

# Save predictions CSV
predictions_df.to_csv(PREDICTIONS_OUT, index=False, float_format='%.6f')

print(f"   Predictions saved: {PREDICTIONS_OUT}")
print(f"   Total predictions: {len(predictions_df):,}")

# Show sample predictions
print(f"\n📋 Sample predictions:")
print(predictions_df.head(10).to_string(index=False))

# ══════════════════════════════════════════════════════════════════════
# SAVE MODEL AND SUMMARY
# ══════════════════════════════════════════════════════════════════════
print(f"\n💾 Saving model and summary...")

# Save model
joblib.dump(model, MODEL_OUT)

# Create comprehensive summary
summary = {
    "timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "model_type": "RandomForest_Optimized",
    "parameters": BEST_PARAMS,
    "dataset_info": {
        "total_samples": len(X),
        "train_samples": len(X_train),
        "test_samples": len(X_test),
        "features": X.shape[1],
        "train_period": f"{df_clean.index[0]} to {df_clean.index[split-1]}",
        "test_period": f"{df_clean.index[split]} to {df_clean.index[-1]}"
    },
    "training_info": {
        "training_time_seconds": train_time,
        "decision_threshold": DECISION_THRESHOLD,
        "oob_score": getattr(model, 'oob_score_', None)
    },
    "performance_metrics": {
        "accuracy": float(accuracy),
        "precision": float(precision),
        "recall": float(recall),
        "f1_score": float(f1),
        "f_beta_score": float(f_beta),
        "auc": float(auc),
        "positive_predictions": int(np.sum(y_pred)),
        "confusion_matrix": cm.tolist()
    },
    "feature_importance": feature_importance.to_dict('records'),
    "class_distribution": {
        "train_positive_rate": float(np.mean(y_train)),
        "test_positive_rate": float(np.mean(y_test)),
        "train_counts": [int(np.sum(y_train == 0)), int(np.sum(y_train == 1))],
        "test_counts": [int(np.sum(y_test == 0)), int(np.sum(y_test == 1))]
    }
}

# Save summary
with open(SUMMARY_JSON, "w") as f:
    json.dump(summary, f, indent=2)

# ══════════════════════════════════════════════════════════════════════
# FINAL REPORT
# ══════════════════════════════════════════════════════════════════════
print(f"\n🎉 Random Forest Final Training Complete!")
print(f"=" * 55)
print(f"📈 Final Performance:")
print(f"   Precision: {precision:.3f} (target: >0.55 for trading)")
print(f"   F-beta:    {f_beta:.3f} (precision-weighted)")
print(f"   Recall:    {recall:.3f} (opportunity capture)")
print(f"   AUC:       {auc:.3f} (overall discrimination)")

print(f"\n📁 Files Generated:")
print(f"   • {MODEL_OUT} - Trained Random Forest model")
print(f"   • {SCALER_OUT} - Feature scaler (for consistency)")
print(f"   • {PREDICTIONS_OUT} - Test predictions ({len(predictions_df):,} rows)")
print(f"   • {SUMMARY_JSON} - Complete training summary")

# Performance assessment
if precision >= 0.55:
    print(f"\n🏆 SUCCESS: Model achieves target precision >0.55!")
    print(f"   Ready for production trading signals")
elif precision >= 0.50:
    print(f"\n⚡ GOOD: Model shows decent precision >0.50")
    print(f"   Consider ensemble with neural networks")
else:
    print(f"\n⚠️  IMPROVEMENT NEEDED: Precision <0.50")
    print(f"   Consider feature engineering or ensemble approach")

print(f"\n🎯 Model ready for production use!")
print(f"   Expected: ~{precision:.0%} precision with ~{recall:.0%} recall")

print(f"\n✨ Random Forest training pipeline completed successfully!")

In [9]:
# =============================================================
#  Random-Forest – batch evaluation of multiple parameter sets
# =============================================================
import numpy as np, pandas as pd, time, sys, warnings, joblib, json, random
from pathlib import Path
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix,
                             classification_report)
warnings.filterwarnings("ignore")
np.random.seed(42); random.seed(42)

# ──────────────────────────────────────────────────────────────
# CONFIG
# ──────────────────────────────────────────────────────────────
CSV_FILE   = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                  r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
TIME_COL   = "timestamp"
TARGET_COL = "target"
START_DATE = "2018-01-01"
TEST_FRAC  = 0.20
THR        = 0.5      # fixed decision threshold
BETA       = 0.5      # F-β with β = 0.5  → precision ×2 weight

# ——— columns to drop (identical to optimisation script) ———
DROP_COLS = [
    'open','high','low','close','high_low','high_close','low_close','typical_price',
    'vwap_24h','close_4h','volume_breakout','volume_breakdown','break_upper_band',
    'break_lower_band','vol_spike_1_5x','rsi_oversold','rsi_overbought',
    'stoch_overbought','stoch_oversold','cci_overbought','cci_oversold',
    'near_upper_band','near_lower_band','overbought_reversal','oversold_reversal',
    'ema_cross_up','ema_cross_down','macd_cross_up','macd_cross_down',
    'trending_market','trend_alignment','ema7_above_ema21','macd_rising',
    'bollinger_upper','bollinger_lower','bollinger_width','resistance_level',
    'support_level','bullish_scenario_1','bullish_scenario_2','bullish_scenario_3',
    'bullish_scenario_4','bullish_scenario_5','bullish_scenario_6',
    'bearish_scenario_1','bearish_scenario_2','bearish_scenario_3',
    'bearish_scenario_4','bearish_scenario_6','EMA_7','EMA_21','SMA_20','SMA_50',
    'MACD_line','MACD_signal','timestamp','date','Unnamed: 0'
]

# ──────────────────────────────────────────────────────────────
# PARAMETER SETS TO TEST
# ──────────────────────────────────────────────────────────────
PARAM_SETS = [
    { "name":"Trial-2-TOP-PRECISION",
      "n_estimators":408,"max_depth":None,"min_samples_split":31,"min_samples_leaf":5,
      "class_weight":"balanced_subsample" },

    { "name":"Trial-71",
      "n_estimators":243,"max_depth":8,"min_samples_split":32,"min_samples_leaf":7,
      "class_weight":"balanced_subsample" },

    { "name":"Trial-99",
      "n_estimators":282,"max_depth":20,"min_samples_split":32,"min_samples_leaf":5,
      "class_weight":"balanced" },

    { "name":"Trial-14-BEST-PRECISION",
      "n_estimators":234,"max_depth":12,"min_samples_split":37,"min_samples_leaf":11,
      "class_weight":"balanced_subsample" },

    { "name":"Trial-25",
      "n_estimators":319,"max_depth":18,"min_samples_split":11,"min_samples_leaf":4,
      "class_weight":"balanced_subsample" },

    { "name":"OPT-WINNER-CV",
      "n_estimators":516,"max_depth":18,"min_samples_split":10,"min_samples_leaf":5,
      "max_features":0.3,"max_leaf_nodes":100,"min_impurity_decrease":0.009414648087765251,
      "class_weight":None },

    { "name":"CUSTOM-HIGH-PRECISION",
      "n_estimators":300,"max_depth":10,"min_samples_split":40,"min_samples_leaf":12,
      "class_weight":"balanced_subsample","max_features":"sqrt" },

    { "name":"CUSTOM-CONSERVATIVE",
      "n_estimators":400,"max_depth":8,"min_samples_split":50,"min_samples_leaf":15,
      "class_weight":"balanced","max_features":0.4 },

    { "name": "BEST-PARAMS-FINAL",
      "n_estimators":300,"max_depth":15,"min_samples_split":10,"min_samples_leaf":4,
      "max_leaf_nodes":200,"max_features":"sqrt","class_weight":"balanced_subsample",
      "max_samples":0.8 },

    { "name": "HIGH-SAMPLE-VARIANT",
      "n_estimators":400,"max_depth":8,"min_samples_split":5,"min_samples_leaf":10,
      "max_leaf_nodes":500,"max_features":0.5,"class_weight":"balanced_subsample",
      "max_samples":0.9 },
]

# defaults merged into every set if a key is missing
DEFAULT_RF_PARAMS = dict(
    max_leaf_nodes       = None,
    max_features         = "sqrt",
    bootstrap            = True,
    random_state         = 42,
    n_jobs               = -1,
    criterion            = "gini",
    min_impurity_decrease= 0.0,
    oob_score            = True
)

# ═════════════════════════════════════════════════════════════
# HELPERS
# ═════════════════════════════════════════════════════════════
def f_beta_05(y_true, y_pred):
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    if p+r == 0: return 0.0
    return (1+BETA**2)*p*r / (BETA**2*p + r)

def evaluate_comprehensive(y_true, y_pred, y_prob):
    cm = confusion_matrix(y_true, y_pred)
    return {
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall':    recall_score(y_true, y_pred, zero_division=0),
        'f1':        f1_score(y_true, y_pred, zero_division=0),
        'f05':       f_beta_05(y_true, y_pred),
        'accuracy':  accuracy_score(y_true, y_pred),
        'auc':       roc_auc_score(y_true, y_prob),
        'tn': cm[0,0], 'fp': cm[0,1], 'fn': cm[1,0], 'tp': cm[1,1],
        'pos_pred': np.sum(y_pred),
        'pos_rate': np.mean(y_pred)
    }

# ═════════════════════════════════════════════════════════════
# 1) LOAD & CLEAN DATA ONCE
# ═════════════════════════════════════════════════════════════
print("🚀 Random Forest Parameter Comparison")
print("=" * 50)

df = (pd.read_csv(CSV_FILE, parse_dates=[TIME_COL])
        .set_index(TIME_COL).sort_index()
        .loc[START_DATE:])

X = df.drop(columns=[c for c in DROP_COLS if c in df.columns] + [TARGET_COL])
y = df[TARGET_COL]
mask = ~(X.isnull().any(axis=1) | y.isnull())
X, y, df = X[mask], y[mask], df[mask]

split = int(len(X)*(1-TEST_FRAC))
X_tr, X_te = X.iloc[:split], X.iloc[split:]
y_tr, y_te = y.iloc[:split], y.iloc[split:]

print(f"   Train: {len(X_tr):,} | Test: {len(X_te):,} | Features: {X_tr.shape[1]}")

# ═════════════════════════════════════════════════════════════
# 2) LOOP OVER PARAM SETS
# ═════════════════════════════════════════════════════════════
print(f"\n🏗️ Training {len(PARAM_SETS)} Random Forest configurations...")
print("=" * 70)

results = []
models  = {}

for i, cfg in enumerate(PARAM_SETS, 1):
    # merge with defaults; “name” kept only for bookkeeping
    full_cfg = {**DEFAULT_RF_PARAMS, **cfg}
    tag      = full_cfg.pop("name")           # remove & store
    model_cfg = full_cfg                      # now safe for RF

    print(f"\n[{i}/{len(PARAM_SETS)}] 🚀 {tag}")
    print("─" * 60)
    shown = {k: v for k, v in model_cfg.items()
             if k not in ("random_state","n_jobs","oob_score","bootstrap","criterion")}
    print("  📋 Params:", ", ".join(f"{k}={v}" for k, v in shown.items()))

    # ── train
    print("  🏗️ Training...", end=" ")
    t0 = time.time()
    model = RandomForestClassifier(**model_cfg)
    model.fit(X_tr, y_tr)
    fit_time = time.time() - t0
    print(f"✅ {fit_time:.1f}s")

    # ── evaluate
    prob = model.predict_proba(X_te)[:,1]
    pred = (prob >= THR).astype(int)
    metrics = evaluate_comprehensive(y_te, pred, prob)
    print(f"  🎯 P={metrics['precision']:.3f} | R={metrics['recall']:.3f} | "
          f"F1={metrics['f1']:.3f} | F0.5={metrics['f05']:.3f} | AUC={metrics['auc']:.3f}")

    # store
    results.append({'name': tag, 'fit_time': fit_time,
                    'oob_score': getattr(model, 'oob_score_', None),
                    **metrics, **shown})
    models[tag] = model

# ═════════════════════════════════════════════════════════════
# 3) LEADERBOARD
# ═════════════════════════════════════════════════════════════
board = pd.DataFrame(results)

print("\n🏆  LEADERBOARD BY F0.5")
print(board.sort_values("f05", ascending=False)
           [["name","precision","recall","f1","f05","auc"]]
           .round(3).to_string(index=False))

print("\n🏆  LEADERBOARD BY PRECISION")
print(board.sort_values("precision", ascending=False)
           [["name","precision","recall","f1","f05","pos_pred"]]
           .round(3).to_string(index=False))

# ═════════════════════════════════════════════════════════════
# 4) SAVE RESULTS & MODELS
# ═════════════════════════════════════════════════════════════
board.to_csv("rf_param_comparison_detailed.csv", index=False)

best_f05   = board.sort_values("f05",   ascending=False).iloc[0]
best_prec  = board.sort_values("precision", ascending=False).iloc[0]

joblib.dump(models[best_f05['name']],  f"rf_best_f05_{best_f05['name']}.joblib")
joblib.dump(models[best_prec['name']], f"rf_best_precision_{best_prec['name']}.joblib")

summary = {
    "timestamp": datetime.utcnow().isoformat(timespec="seconds")+"Z",
    "best_by_f05":   best_f05.to_dict(),
    "best_by_prec":  best_prec.to_dict(),
    "all_results":   board.to_dict('records')
}
with open("rf_comparison_summary.json","w") as fp:
    json.dump(summary, fp, indent=2)

print("\n✨  Parameter comparison finished. Detailed CSV, summary JSON "
      "and the top models have been saved.")


🚀 Random Forest Parameter Comparison
   Train: 12,684 | Test: 3,171 | Features: 26

🏗️ Training 10 Random Forest configurations...

[1/10] 🚀 Trial-2-TOP-PRECISION
────────────────────────────────────────────────────────────
  📋 Params: max_leaf_nodes=None, max_features=sqrt, min_impurity_decrease=0.0, n_estimators=408, max_depth=None, min_samples_split=31, min_samples_leaf=5, class_weight=balanced_subsample
  🏗️ Training... ✅ 2.8s
  🎯 P=0.581 | R=0.354 | F1=0.440 | F0.5=0.515 | AUC=0.548

[2/10] 🚀 Trial-71
────────────────────────────────────────────────────────────
  📋 Params: max_leaf_nodes=None, max_features=sqrt, min_impurity_decrease=0.0, n_estimators=243, max_depth=8, min_samples_split=32, min_samples_leaf=7, class_weight=balanced_subsample
  🏗️ Training... ✅ 1.2s
  🎯 P=0.576 | R=0.360 | F1=0.443 | F0.5=0.514 | AUC=0.549

[3/10] 🚀 Trial-99
────────────────────────────────────────────────────────────
  📋 Params: max_leaf_nodes=None, max_features=sqrt, min_impurity_decrease=0.0, n_

In [10]:
# =============================================================
#  Random Forest - CUSTOM-HIGH-PRECISION Final Training
# =============================================================
import numpy as np, pandas as pd, time, sys, warnings, joblib, json
from pathlib import Path
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix,
                             classification_report)
warnings.filterwarnings("ignore")
np.random.seed(42)

# ──────────────────────────────────────────────────────────────
# CONFIG
# ──────────────────────────────────────────────────────────────
CSV_FILE   = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                  r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
TIME_COL   = "timestamp"
TARGET_COL = "target"
START_DATE = "2018-01-01"
TEST_FRAC  = 0.20
THR        = 0.5      # decision threshold
BETA       = 0.5      # F-beta weighting

# Output files
MODEL_FILE = "rf_custom_high_precision_final.joblib"
PREDICTIONS_CSV = "rf_predictions_custom_high_precision.csv"
SUMMARY_JSON = "rf_training_summary_custom_high_precision.json"

# ——— columns to drop (identical to optimization script) ———
DROP_COLS = [
    'open','high','low','close','high_low','high_close','low_close','typical_price',
    'vwap_24h','close_4h','volume_breakout','volume_breakdown','break_upper_band',
    'break_lower_band','vol_spike_1_5x','rsi_oversold','rsi_overbought',
    'stoch_overbought','stoch_oversold','cci_overbought','cci_oversold',
    'near_upper_band','near_lower_band','overbought_reversal','oversold_reversal',
    'ema_cross_up','ema_cross_down','macd_cross_up','macd_cross_down',
    'trending_market','trend_alignment','ema7_above_ema21','macd_rising',
    'bollinger_upper','bollinger_lower','bollinger_width','resistance_level',
    'support_level','bullish_scenario_1','bullish_scenario_2','bullish_scenario_3',
    'bullish_scenario_4','bullish_scenario_5','bullish_scenario_6',
    'bearish_scenario_1','bearish_scenario_2','bearish_scenario_3',
    'bearish_scenario_4','bearish_scenario_6','EMA_7','EMA_21','SMA_20','SMA_50',
    'MACD_line','MACD_signal','timestamp','date','Unnamed: 0'
]

# ──────────────────────────────────────────────────────────────
# CUSTOM-HIGH-PRECISION PARAMETERS (your best config)
# ──────────────────────────────────────────────────────────────
BEST_PARAMS = {
    "n_estimators": 300,
    "max_depth": 10,
    "min_samples_split": 40,
    "min_samples_leaf": 12,
    "class_weight": "balanced_subsample",
    "max_features": "sqrt",
    "bootstrap": True,
    "random_state": 42,
    "n_jobs": -1,
    "criterion": "gini",
    "min_impurity_decrease": 0.0,
    "oob_score": True,
    "warm_start": False,
    "verbose": 0
}

# ═════════════════════════════════════════════════════════════
# HELPER FUNCTIONS
# ═════════════════════════════════════════════════════════════
def f_beta_05(y_true, y_pred):
    """F-beta score with beta=0.5 (precision-weighted)"""
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    if p + r == 0: 
        return 0.0
    return (1 + BETA**2) * p * r / (BETA**2 * p + r)

def evaluate_model(y_true, y_pred, y_prob):
    """Comprehensive model evaluation"""
    cm = confusion_matrix(y_true, y_pred)
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1_score': f1_score(y_true, y_pred, zero_division=0),
        'f_beta': f_beta_05(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_prob),
        'confusion_matrix': cm,
        'tn': cm[0,0], 'fp': cm[0,1], 'fn': cm[1,0], 'tp': cm[1,1],
        'positive_predictions': np.sum(y_pred),
        'positive_rate': np.mean(y_pred)
    }

# ═════════════════════════════════════════════════════════════
# MAIN TRAINING PIPELINE
# ═════════════════════════════════════════════════════════════
print("🚀 CUSTOM-HIGH-PRECISION Random Forest Training")
print("=" * 55)
print("🎯 Target: High precision for trading signals")
print("📊 Expected performance: ~0.581 precision")

# ──────────────────────────────────────────────────────────────
# 1) LOAD & PREPARE DATA
# ──────────────────────────────────────────────────────────────
print(f"\n📂 Loading data from: {CSV_FILE.name}")
if not CSV_FILE.exists():
    sys.exit(f"❌ File not found: {CSV_FILE}")

# Load and clean data
df = (pd.read_csv(CSV_FILE, parse_dates=[TIME_COL])
        .set_index(TIME_COL).sort_index()
        .loc[START_DATE:])

print(f"   📅 Date range: {df.index.min()} to {df.index.max()}")
print(f"   📊 Raw data shape: {df.shape}")

# Feature engineering
X = df.drop(columns=[c for c in DROP_COLS if c in df.columns] + [TARGET_COL])
y = df[TARGET_COL]

# Remove NaN values
initial_size = len(X)
mask = ~(X.isnull().any(axis=1) | y.isnull())
X, y, df_clean = X[mask], y[mask], df[mask]

print(f"   🧹 Cleaned data: {len(X):,} samples ({initial_size - len(X)} removed)")
print(f"   🎯 Features: {X.shape[1]} | Target balance: {y.mean():.1%} bullish")

# Show features being used
print(f"   ✅ Features: {list(X.columns)}")

# Time-based split (crucial for financial data)
split = int(len(X) * (1 - TEST_FRAC))
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

print(f"\n📈 Time-based Train/Test Split:")
print(f"   Train: {len(X_train):,} samples ({df_clean.index[0]} to {df_clean.index[split-1]})")
print(f"   Test:  {len(X_test):,} samples ({df_clean.index[split]} to {df_clean.index[-1]})")
print(f"   Train target rate: {y_train.mean():.3f}")
print(f"   Test target rate:  {y_test.mean():.3f}")

# ──────────────────────────────────────────────────────────────
# 2) TRAIN CUSTOM-HIGH-PRECISION MODEL
# ──────────────────────────────────────────────────────────────
print(f"\n🏗️ Training CUSTOM-HIGH-PRECISION Random Forest...")
print("─" * 60)

# Display key parameters
print(f"   📋 Model Configuration:")
print(f"      n_estimators: {BEST_PARAMS['n_estimators']}")
print(f"      max_depth: {BEST_PARAMS['max_depth']}")
print(f"      min_samples_split: {BEST_PARAMS['min_samples_split']}")
print(f"      min_samples_leaf: {BEST_PARAMS['min_samples_leaf']}")
print(f"      class_weight: {BEST_PARAMS['class_weight']}")
print(f"      max_features: {BEST_PARAMS['max_features']}")

print(f"\n🚀 Training model...", end=" ", flush=True)
start_time = time.time()

# Train the model
model = RandomForestClassifier(**BEST_PARAMS)
model.fit(X_train, y_train)

training_time = time.time() - start_time
print(f"✅ Completed in {training_time:.1f}s")

# ──────────────────────────────────────────────────────────────
# 3) EVALUATE MODEL PERFORMANCE
# ──────────────────────────────────────────────────────────────
print(f"\n📊 Evaluating model performance...")

# Generate predictions
y_prob = model.predict_proba(X_test)[:, 1]  # probability of class 1 (up)
y_pred = (y_prob >= THR).astype(int)

# Calculate metrics
metrics = evaluate_model(y_test, y_pred, y_prob)

print(f"\n🎯 PERFORMANCE RESULTS:")
print("=" * 50)
print(f"   Accuracy:               {metrics['accuracy']:.4f}")
print(f"   Precision:              {metrics['precision']:.4f} ⭐")
print(f"   Recall:                 {metrics['recall']:.4f}")
print(f"   F1 Score:               {metrics['f1_score']:.4f}")
print(f"   F-beta (β=0.5):         {metrics['f_beta']:.4f}")
print(f"   ROC AUC:                {metrics['roc_auc']:.4f}")
print(f"   Positive predictions:   {metrics['positive_predictions']:,} / {len(y_test):,}")
print(f"   Positive rate:          {metrics['positive_rate']:.1%}")

# Confusion Matrix
print(f"\n📋 Confusion Matrix:")
print(f"   True Negatives (TN):  {metrics['tn']:,}")
print(f"   False Positives (FP): {metrics['fp']:,}")
print(f"   False Negatives (FN): {metrics['fn']:,}")
print(f"   True Positives (TP):  {metrics['tp']:,}")

# OOB Score
if hasattr(model, 'oob_score_'):
    print(f"\n🔄 Out-of-Bag Score: {model.oob_score_:.4f}")

# Classification Report
print(f"\n📋 Detailed Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Down", "Up"]))

# ──────────────────────────────────────────────────────────────
# 4) FEATURE IMPORTANCE ANALYSIS
# ──────────────────────────────────────────────────────────────
print(f"\n🌟 Feature Importance Analysis...")

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🌟 TOP 10 MOST IMPORTANT FEATURES:")
print("─" * 50)
for i, (_, row) in enumerate(feature_importance.head(10).iterrows(), 1):
    print(f"   {i:2d}. {row['feature']:<25}: {row['importance']:.4f}")

# Feature importance insights
top_5_importance = feature_importance.head(5)['importance'].sum()
top_10_importance = feature_importance.head(10)['importance'].sum()

print(f"\n📊 Feature Concentration:")
print(f"   Top 5 features:  {top_5_importance:.1%} of model decisions")
print(f"   Top 10 features: {top_10_importance:.1%} of model decisions")

# ──────────────────────────────────────────────────────────────
# 5) GENERATE PREDICTIONS CSV IN EXACT FORMAT
# ──────────────────────────────────────────────────────────────
print(f"\n📁 Generating predictions CSV...")

# Calculate probabilities and predictions
prob_up = y_prob
prob_down = 1.0 - prob_up
winning_prob = np.maximum(prob_up, prob_down)

# Create predictions DataFrame in the EXACT format you specified
predictions_df = pd.DataFrame({
    'timestamp': X_test.index.strftime('%Y-%m-%d %H:%M:%S'),
    'prob_up': prob_up,
    'prob_down': prob_down,
    'winning_prob': winning_prob,
    'prediction': y_pred,
    'actual': y_test.values
})

# Save with exact formatting
predictions_df.to_csv(PREDICTIONS_CSV, index=False, float_format='%.6f')

print(f"   ✅ Predictions saved: {PREDICTIONS_CSV}")
print(f"   📊 Total predictions: {len(predictions_df):,}")

# Show sample of the exact format
print(f"\n📋 Sample predictions (your exact format):")
sample_predictions = predictions_df.head(6)
for _, row in sample_predictions.iterrows():
    print(f"   {row['timestamp']} {row['prob_up']:.6f} {row['prob_down']:.6f} "
          f"{row['winning_prob']:.6f} {row['prediction']} {row['actual']}")

# ──────────────────────────────────────────────────────────────
# 6) SAVE MODEL AND SUMMARY
# ──────────────────────────────────────────────────────────────
print(f"\n💾 Saving model and comprehensive summary...")

# Save the trained model
joblib.dump(model, MODEL_FILE)
print(f"   🏆 Model saved: {MODEL_FILE}")

# Create comprehensive training summary
summary = {
    "model_info": {
        "model_name": "CUSTOM-HIGH-PRECISION",
        "model_type": "RandomForestClassifier",
        "training_timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "training_time_seconds": training_time,
        "parameters": BEST_PARAMS
    },
    "dataset_info": {
        "total_samples": len(X),
        "train_samples": len(X_train),
        "test_samples": len(X_test),
        "features": X.shape[1],
        "feature_list": list(X.columns),
        "train_period": f"{df_clean.index[0]} to {df_clean.index[split-1]}",
        "test_period": f"{df_clean.index[split]} to {df_clean.index[-1]}",
        "target_balance": {
            "train_positive_rate": float(y_train.mean()),
            "test_positive_rate": float(y_test.mean())
        }
    },
    "performance_metrics": {
        "decision_threshold": THR,
        "accuracy": float(metrics['accuracy']),
        "precision": float(metrics['precision']),
        "recall": float(metrics['recall']),
        "f1_score": float(metrics['f1_score']),
        "f_beta_score": float(metrics['f_beta']),
        "roc_auc": float(metrics['roc_auc']),
        "positive_predictions": int(metrics['positive_predictions']),
        "positive_rate": float(metrics['positive_rate']),
        "oob_score": float(getattr(model, 'oob_score_', 0)),
        "confusion_matrix": {
            "true_negatives": int(metrics['tn']),
            "false_positives": int(metrics['fp']),
            "false_negatives": int(metrics['fn']),
            "true_positives": int(metrics['tp'])
        }
    },
    "feature_importance": feature_importance.to_dict('records'),
    "files_generated": {
        "model_file": MODEL_FILE,
        "predictions_csv": PREDICTIONS_CSV,
        "summary_json": SUMMARY_JSON
    }
}

# Save summary
with open(SUMMARY_JSON, "w") as f:
    json.dump(summary, f, indent=2)

print(f"   📋 Summary saved: {SUMMARY_JSON}")

# ──────────────────────────────────────────────────────────────
# 7) FINAL ASSESSMENT AND RECOMMENDATIONS
# ──────────────────────────────────────────────────────────────
print(f"\n🎉 CUSTOM-HIGH-PRECISION Training Complete!")
print("=" * 60)

final_precision = metrics['precision']
final_f_beta = metrics['f_beta']

print(f"\n📊 FINAL PERFORMANCE ASSESSMENT:")
if final_precision >= 0.57:
    print(f"   🏆 EXCEPTIONAL: {final_precision:.3f} precision - Elite trading model!")
    print(f"   ✅ Ready for live trading with high confidence")
elif final_precision >= 0.55:
    print(f"   🥇 EXCELLENT: {final_precision:.3f} precision - Strong trading candidate")
    print(f"   ✅ Recommended for production use")
elif final_precision >= 0.52:
    print(f"   ⚡ GOOD: {final_precision:.3f} precision - Viable with risk management")
    print(f"   ⚠️  Consider position sizing adjustments")
else:
    print(f"   📈 DEVELOPING: {final_precision:.3f} precision - Monitor performance")
    print(f"   💡 Consider ensemble or parameter fine-tuning")

print(f"\n🎯 KEY METRICS ACHIEVED:")
print(f"   • Precision:     {final_precision:.3f} (target: >0.55)")
print(f"   • F-beta (β=0.5): {final_f_beta:.3f} (precision-weighted)")
print(f"   • ROC AUC:       {metrics['roc_auc']:.3f} (discrimination power)")

print(f"\n📁 OUTPUT FILES:")
print(f"   • {MODEL_FILE} - Trained model ready for production")
print(f"   • {PREDICTIONS_CSV} - Test predictions in your exact format")
print(f"   • {SUMMARY_JSON} - Complete training documentation")

print(f"\n✨ Model is ready for production trading!")
print(f"🎯 Expected trading precision: ~{final_precision:.1%}")
print(f"🚀 Use this model to generate high-confidence trading signals!")

🚀 CUSTOM-HIGH-PRECISION Random Forest Training
🎯 Target: High precision for trading signals
📊 Expected performance: ~0.581 precision

📂 Loading data from: gemini_btc_with_features_4h.csv
   📅 Date range: 2018-01-01 00:00:00 to 2025-03-28 00:00:00
   📊 Raw data shape: (15855, 66)
   🧹 Cleaned data: 15,855 samples (0 removed)
   🎯 Features: 26 | Target balance: 51.1% bullish
   ✅ Features: ['volume', 'RSI', 'MACD_histogram', 'OBV', 'CCI', 'stoch_%K', 'stoch_%D', 'true_range', 'atr_14', 'atr_ratio', 'parkinson_vol', 'price_vs_vwap', 'volume_mean_20', 'volume_ratio', 'buying_pressure', 'adx', 'volatility_regime', 'fear_greed_score', 'roc_4h', 'roc_24h', 'bb_position', 'above_sma20', 'above_sma50', 'macd_positive', 'obv_rising_24h', 'momentum_alignment']

📈 Time-based Train/Test Split:
   Train: 12,684 samples (2018-01-01 00:00:00 to 2023-10-16 12:00:00)
   Test:  3,171 samples (2023-10-16 16:00:00 to 2025-03-28 00:00:00)
   Train target rate: 0.508
   Test target rate:  0.522

🏗️ Training 

In [ ]:
"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models\rf_predictions_custom_high_precision.csv"

In [11]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the predictions CSV
csv_path = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\src\Models\models\models\rf_predictions_custom_high_precision.csv"
df = pd.read_csv(csv_path)

# Ensure column names are correct and lowercase
df.columns = df.columns.str.strip().str.lower()

# Extract actual and predicted values
y_true = df['actual']
y_pred = df['prediction']  # prediction at threshold 0.5

# Calculate metrics
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Print results
print("📊 Evaluation at threshold 0.5:")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1 Score : {f1:.3f}")


📊 Evaluation at threshold 0.5:
Precision: 0.581
Recall   : 0.363
F1 Score : 0.447
